In [1]:
import pandas as pd

In [2]:
testCSV = pd.read_csv('Dataset(Analysis)(processed lyrics).csv')
df= pd.DataFrame(testCSV)
print(sum(df["Hit"] == 1))
print(sum(df["Hit"] == 0))

351
352


In [3]:
from sklearn.feature_extraction.text import CountVectorizer
bow = CountVectorizer(max_features=1000,
                      lowercase=True,
                      ngram_range=(1,1),
                      analyzer = "word").fit(testCSV['Lyrics'].values.astype(str))
print(len(bow.vocabulary_))

1000


In [4]:
lyrics_bow = bow.transform(testCSV['Lyrics'].values.astype(str)).todense()
print('Shape of Sparse Matrix: ', lyrics_bow.shape)
print(type(lyrics_bow))
# lyrics_bow.nnz

Shape of Sparse Matrix:  (703, 1000)
<class 'numpy.matrixlib.defmatrix.matrix'>


In [5]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer().fit(lyrics_bow)
lyrics_tfidf = tfidf_transformer.transform(lyrics_bow)

In [6]:
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB().fit(lyrics_bow, testCSV['Hit'])

from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(lyrics_bow, testCSV['Hit'], test_size=0.2,random_state = 938)
model.fit(X_train, Y_train)
predictions = model.predict(X_test)

In [7]:
predictions

array([1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1,
       0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1,
       1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0,
       1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0,
       1, 0, 0, 0, 1, 1, 0, 1, 1], dtype=int64)

In [8]:
from sklearn.metrics import classification_report
from sklearn.metrics import recall_score
print(recall_score(Y_test,predictions))
print(classification_report(Y_test, predictions))

0.8055555555555556
             precision    recall  f1-score   support

          0       0.71      0.51      0.59        69
          1       0.63      0.81      0.71        72

avg / total       0.67      0.66      0.65       141



In [9]:
from sklearn.metrics import confusion_matrix
cnf_matrix = confusion_matrix(Y_test, predictions)
cnf_matrix

array([[35, 34],
       [14, 58]], dtype=int64)

In [10]:
from sklearn.linear_model import LogisticRegression

In [11]:
modelLog = LogisticRegression()

modelLog.fit(X_train, Y_train)
predictionsLog = modelLog.predict(X_test)

from sklearn.metrics import classification_report
print(classification_report(Y_test, predictionsLog))

             precision    recall  f1-score   support

          0       0.72      0.59      0.65        69
          1       0.67      0.78      0.72        72

avg / total       0.69      0.69      0.69       141



In [12]:
from sklearn.metrics import confusion_matrix
cnf_matrix = confusion_matrix(Y_test, predictionsLog)
cnf_matrix

array([[41, 28],
       [16, 56]], dtype=int64)

# Checking best results by varying random_state in splitting dataset

In [13]:
from sklearn.metrics import classification_report
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score

recall = []
precision=[]
f1 = []
for i in range(0, 1001):
    X_train, X_test, Y_train, Y_test = train_test_split(lyrics_bow, testCSV['Hit'], test_size=0.2,random_state = i)
    model.fit(X_train, Y_train)
    predictions = model.predict(X_test)
    precision.append(precision_score(Y_test,predictions))
    recall.append(recall_score(Y_test,predictions))
    f1.append(f1_score(Y_test,predictions))
    
    

In [14]:
print(max(recall))
print(recall.index(max(recall)))
print('recall at 524: ',  recall[524])

0.8055555555555556
938
recall at 524:  0.6216216216216216


In [15]:
print(precision.index(max(precision)))
print('Precision: ' , max(precision))
print('precision at 291 ', precision[291])

600
Precision:  0.7571428571428571
precision at 291  0.5555555555555556


In [16]:
print('min f1 score index: ',f1.index(min(f1)))
print('min f1 score: ', min(f1))
print(max(f1))
print('f1 score at 291: ',f1[291])

min f1 score index:  945
min f1 score:  0.47692307692307695
0.7283950617283949
f1 score at 291:  0.608108108108108


In [17]:
l = []
for i in range(0, 1001):
    a = [recall[i], precision[i]]
    l.append(a)

In [18]:
max(l)

[0.8055555555555556, 0.6304347826086957]